## BioImage.IO Chatbot Extensions

Documentation: https://github.com/bioimage-io/bioimageio-chatbot/blob/main/docs/development.md

### Using the chatbot in an ImJoy plugin

Below you will find an example of how to use the chatbot in an ImJoy plugin. Using `api.createWindow`, we can display the chatbot inline as an imjoy plugin and interact with its api.

In [25]:
from imjoy import api

async def setup():
    chatbot = await api.createWindow(
        src="https://bioimage.io/chat",
        name="BioImage.IO Chatbot",
    )
    
    await api.showMessage(str(await chatbot.getAllExtensions()))

api.export({"setup": setup})

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

## Create a chatbot extension

A chatbot extension object is a dictionary with the following keys:
 - `id`: a unique identifier for the extension
 - `name`: the name of the extension
 - `description`: a short description of the extension
 - `type`: it must be `bioimageio-chatbot-extension`
 - `tools`: a dictionary with functions of tools, it represents the set of functions your extension offers, each accepting configuration parameters as input. These functions should carry out specific tasks and return their results in a dictionary.
 - `get_schema`: a function returns the schema for the tools, it returns a JSON schema for each tool function, specifying the structure and types of the expected parameters. This schema is crucial for instructing the chatbot to generate the correct input paramters and validate the inputs and ensuring they adhere to the expected format. Importantly, the chatbot uses the title and description for each field to understand what expected for the tool will generating a function call to run the tool (also see the detailed instructions below).


 The following is an example of creating a chatbot extension:

In [ ]:
from pydantic import BaseModel, Field
from skimage import io, filters, data


class ApplyFilterInput(BaseModel):
    """Apply a gaussian filter to an example image"""
    sigma: float = Field(..., description="Standard deviation for Gaussian kernel")

async def apply_filter(kwargs):
    config = ApplyFilterInput(**kwargs)

    # Load the image
    image = data.cell()

    # Apply the filter
    filtered_image = filters.gaussian(image, sigma=config.sigma)

    # Display the image
    viewer = await api.showDialog(src="https://kitware.github.io/itk-vtk-viewer/app/")
    await viewer.setImage(filtered_image)
    
    return "Filter applied successfully"

def get_schema():
    return {
        "apply_filter": ApplyFilterInput.schema(),
    }

# Define a chatbot extension
image_processing_extension = {
    "_rintf": True,
    "id": "image-processing",
    "type": "bioimageio-chatbot-extension",
    "name": "Image Processing",
    "description": "Apply gaussian filter to an example image",
    "get_schema": get_schema,
    "tools": {
        "apply_filter": apply_filter,
    }
}

## Option 1: Register a chatbot extension as an ImJoy plugin

The following will show the chatbot window, then you can type for example `run the gaussian filter example` to invoke the tool you defined in the extension.

In [ ]:
from imjoy_rpc import api

async def setup():
    chatbot = await api.createWindow(src="https://bioimage.io/chat", name="BioImage.IO Chatbot")
    await chatbot.registerExtension(image_processing_extension)

api.export({"setup": setup})

## Option 2: Serving extension remotely using hypha

Besides running the extension in the browser using ImJoy, you can also run the extension remotely using [hypha](https://ha.amun.ai). This allows the extension to run in a native Python environment which have easier access to hardware devices (e.g. the actual microscope) and more computational resources.

Similar to the ImJoy plugin, you need to register the extension as a hypha service.

In [ ]:
from imjoy_rpc.hypha import connect_to_server, login

server_url = "https://chat.bioimage.io"
token = await login({"server_url": server_url})
server = await connect_to_server({"server_url": server_url, "token": token})
# Below, we set the visibility to public
image_processing_extension['config'] = {"visibility": "public"}
svc = await server.register_service(image_processing_extension)
print(f"Extension service registered with id: {svc.id}, you can visit the service at: https://bioimage.io/chat?server={server_url}&extension={svc.id}&assistant=Skyler")

To use the chatbot, you can now click the link above, or use the following code to run the chatbot in the browser.

In [ ]:
## Display the chatbot as embedded iframe
from IPython.display import display, IFrame
display(IFrame(src="https://bioimage.io/chat", width="100%", height="600px"))

## Additional Examples
### Example 1: Controlling a microscope using the chatbot

In [ ]:
from pydantic import BaseModel, Field
    
class MoveStageInput(BaseModel):
    """Move the microscope stage"""
    x: float = Field(..., description="x offset")
    y: float = Field(..., description="y offset")

class SnapImageInput(BaseModel):
    """Move the microscope stage"""
    exposure: float = Field(..., description="exposure time")

async def move_stage(kwargs):
    config = MoveStageInput(**kwargs)
    print(config.x, config.y)

    return "success"

async def snap_image(kwargs):
    config = SnapImageInput(**kwargs)
    print(config.exposure)
    await api.showDialog(src="https://bioimage.io")
    return "success"

def get_schema():
        return {
            "move_stage": MoveStageInput.schema(),
            "snap_image": SnapImageInput.schema()
        }

# Define an chatbot extension
microscope_control_extension = {
    "_rintf": True,
    "id": "microscope-control",
    "type": "bioimageio-chatbot-extension",
    "name": "Microscope Control",
    "description": "Contorl the microscope....",
    "get_schema": get_schema,
    "tools": {
        "move_stage": move_stage,
        "snap_image": snap_image,
    }
}

from imjoy_rpc.hypha import connect_to_server, login

server_url = "https://chat.bioimage.io"
token = await login({"server_url": server_url})
server = await connect_to_server({"server_url": server_url, "token": token})
# Below, we set the visibility to public
microscope_control_extension['config'] = {"visibility": "public"}
svc = await server.register_service(microscope_control_extension)
print(f"Extension service registered with id: {svc.id}, you can visit the service at: https://bioimage.io/chat?server={server_url}&extension={svc.id}&assistant=Skyler")

### Example 2: Query the models in the model zoo via code generation

In [24]:
from imjoy_rpc import api
import sys
import io
from imjoy import api
from js import fetch
from pydantic import BaseModel, Field
from typing import Optional
from typing import List, Optional, Dict, Any

class ResourceType(str):
    MODEL = "model"
    DATASET = "dataset"
    APPLICATION = "application"

def normalize_text(text: str) -> str:
    return text.replace('_', ' ').lower()

def matches_keywords(text: str, keywords: List[str]) -> bool:
    normalized_text = normalize_text(text)
    return any(keyword in normalized_text for keyword in keywords)

def search_item(item: Dict[str, Any], keywords: List[str]) -> bool:
    search_fields = [item.get('id', ''), item.get('nickname', ''), item.get('name', ''),
        item.get('nickname_icon', ''), item.get('license', ''), item.get('description', '')
    ] + [tag for tag in item.get('tags', [])]
    search_fields += [author['name'] for author in item.get('authors', [])]
    return any(matches_keywords(field, keywords) for field in search_fields)

def search(keywords, type, top_k, resource_items: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    keywords = [normalize_text(keyword) for keyword in keywords]
    filtered_items = []
    for item in resource_items:
        if type and item.get('type') != type:
            continue
        if search_item(item, keywords):
            filtered_items.append(item)
        if len(filtered_items) == top_k:
            break
    return filtered_items

async def load_model_info():
    response = await fetch("https://bioimage-io.github.io/collection-bioimage-io/collection.json")
    model_info = await response.json()
    model_info = model_info.to_py()
    resource_items = model_info['collection']
    return resource_items

def execute_code(script, context=None):
    if context is None:
        context = {}

    # Redirect stdout and stderr to capture their output
    original_stdout = sys.stdout
    original_stderr = sys.stderr
    sys.stdout = io.StringIO()
    sys.stderr = io.StringIO()

    try:
        # Create a copy of the context to avoid modifying the original
        local_vars = context.copy()

        # Execute the provided Python script with access to context variables
        exec(script, local_vars)

        # Capture the output from stdout and stderr
        stdout_output = sys.stdout.getvalue()
        stderr_output = sys.stderr.getvalue()

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            # "context": local_vars  # Include context variables in the result
        }
    except Exception as e:
        return {
            "stdout": "",
            "stderr": str(e),
            # "context": context  # Include context variables in the result even if an error occurs
        }
    finally:
        # Restore the original stdout and stderr
        sys.stdout = original_stdout
        sys.stderr = original_stderr

async def register_chatbot_extension(register):
    resource_items = await load_model_info()
    types = set()
    tags = set()
    for resource in resource_items:
        types.add(resource['type'])
        tags.update(resource['tags'])
    types = list(types)
    tags = list(tags)[:5]
    resource_item_stats = f"""- keys: {list(resource_items[0].keys())}\n- resource types: {types}\n- Exampletags: {tags}\n""" #Here is an example: {resource_items[0]}

    class ModelZooInfoScript(BaseModel):
        script: str = Field(..., description="""Executable python script (Python runtime: Pyodide) for querying information""")
    
    ModelZooInfoScript.__doc__ = (
        "Search the BioImage Model Zoo for statistical information by executing Python3 scripts on the resource items."
        "For exampling counting models, applications, and datasets filtered by tags in the BioImage Model Zoo (bioimage.io). "
        "The generated scripts will be executed browser pyodide environment, the script can access data through the 'resources' local variable, containing zoo resources as dictionaries. "
        "Handle any missing fields in zoo items, and ensure outputs are directed to stdout. "
        "Filter resources by the 'type' key without making remote server requests. 'resources' variable details:\\n"
    ) + resource_item_stats


    class ModelZooSearchInput(BaseModel):
        """Search the BioImage Model Zoo (bioimage.io) resource items such as models, applications, datasets, etc. in the model zoo and return detailed information. The link format to the models etc. is: https://bioimage.io/#/?id=[ResourceID]"""
        keywords: List[str] = Field(..., description="List of keywords to search for in the model zoo.")
        top_k: int = Field(3, description="The maximum number of search results to return. Default is 3. Please be aware each item may contain a large amount of data.")
        type: Optional[ResourceType] = Field(None, description="The type of resource to search for. Options include 'model', 'dataset', 'application'.")


    def get_schema():
        return {
            "run_script": ModelZooInfoScript.schema(),
            "search": ModelZooSearchInput.schema()
        }

    async def execute_script(kwargs):
        info_script = ModelZooInfoScript.parse_obj(kwargs)
        result = execute_code(info_script.script, {"resources": resource_items})
        return result

    async def execute_search(kwargs):
        config = ModelZooSearchInput.parse_obj(kwargs)
        result = search(config.keywords, config.type, config.top_k, resource_items)
        return result

    await register({
        "_rintf": True,
        "id": "bioimage_model_zoo",
        "type": "bioimageio-chatbot-extension",
        "name": "BioImage Model Zoo",
        "description": "Getting information about models, applications, datasets, etc. in the BioImage Model Zoo. It takes a list of keywords or a python script to query the resources in the BioImage Model Zoo.",
        "get_schema": get_schema,
        "tools": {
            "run_script": execute_script,
            "search": execute_search,
        }
    })


async def setup():
    chatbot = await api.createWindow(src="https://bioimage.io/chat")
    await register_chatbot_extension(chatbot.registerExtension)

api.export({"setup": setup})

<IPython.core.display.Javascript object>

<_GatheringFuture pending>